# Training a model
see README.md for details

Loosely following this tutorial:
https://medium.com/nlplanet/a-full-guide-to-finetuning-t5-for-text2text-and-building-a-demo-with-streamlit-c72009631887

In [1]:
import torch
import pandas as pd
from datasets import load_dataset, Dataset
from evaluate import load
from transformers import AutoModelForSeq2SeqLM
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer
from tqdm import tqdm
from glob import glob

from config import data_root, model_root, checkpoint_name
from config import token_len, annot_len

In [2]:
torch.cuda.is_available()

False

In [3]:
num_special_tokens = 3
model_checkpoint = "google/byt5-small"

In [12]:
help(Dataset.from_pandas)

Help on method from_pandas in module datasets.arrow_dataset:

from_pandas(df: pandas.core.frame.DataFrame, features: Optional[datasets.features.features.Features] = None, info: Optional[datasets.info.DatasetInfo] = None, split: Optional[datasets.splits.NamedSplit] = None, preserve_index: Optional[bool] = None) -> 'Dataset' method of builtins.type instance
    Convert `pandas.DataFrame` to a `pyarrow.Table` to create a [`Dataset`].
    
    The column types in the resulting Arrow Table are inferred from the dtypes of the `pandas.Series` in the
    DataFrame. In the case of non-object Series, the NumPy dtype is translated to its Arrow equivalent. In the
    case of `object`, we need to guess the datatype by looking at the Python objects in this Series.
    
    Be aware that Series of the `object` dtype don't carry enough information to always lead to a meaningful Arrow
    type. In the case that we cannot infer a type, e.g. because the DataFrame is of length 0 or the Series only
    con

In [16]:
dfs = []
for fname in glob(f"{data_root}/*.csv"):
    dfs += [pd.read_csv(fname, names=["train","test"])]
df = pd.concat(dfs, axis=0)
ds = Dataset.from_pandas(df)
# ds = load_dataset("csv", data_files=glob(f"{data_root}/*.csv"), features=["input","output"])
ds.features

{'train': Value(dtype='string', id=None),
 'test': Value(dtype='string', id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [17]:
exact_match_metric = load("exact_match")

args = Seq2SeqTrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    torch_empty_cache_steps=100,
)


def model_init():
    return AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)


trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=args,
    train_dataset=ds["train"],
    eval_dataset=ds["validation"],
    #     data_collator=data_collator,
    compute_metrics=exact_match_metric,
)

TypeError: Seq2SeqTrainingArguments.__init__() got an unexpected keyword argument 'torch_empty_cache_steps'